In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import os 
from saved_tracks_extractor import get_audio_features_and_analysis, get_genres
from dotenv import load_dotenv

# Configura le tue credenziali Spotify
load_dotenv()
client_id = os.getenv('YOUR_CLIENT_ID')
client_secret = os.getenv('YOUR_CLIENT_SECRET')
redirect_uri = os.getenv('YOUR_REDIRECT_URI')
scope = "user-library-read playlist-modify-public"

# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
#                                                client_secret=client_secret,
#                                                redirect_uri=redirect_uri,
#                                                scope=scope))


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [2]:
target_user_id = '31ighcy6cpa3jsteh7imcasje2ea'

In [3]:
# Initialize an empty list to store the results
track_data = {}

# Set the batch size (max 50 for Spotify API)
batch_size = 20

# Get the total number of saved tracks
total_tracks = sp.current_user_saved_tracks(limit=1)['total']
print(f"Total tracks: {total_tracks}")

total_tracks = 60
# Loop through batches of saved tracks
for offset in range(0, total_tracks, batch_size):
    tracks = sp.current_user_saved_tracks(limit=batch_size, offset=offset)
    track_ids = [item['track']['id'] for item in tracks['items']]
    print(f"Track IDs: {track_ids}")    
    # Apply the get_audio_features_and_analysis function
    track_data_old = get_genres(track_ids, sp)
    # Append the result to the list
    if offset==0:
        track_data=track_data_old
    else:
        for key in track_data:
            track_data[key]+=track_data_old[key]
    
print(track_data.keys())
# Print or process the results as needed
# for result in results:
#     print("---")
#     print(json.dumps(result, indent=4))

Total tracks: 100
Track IDs: ['4o6BgsqLIBViaGVbx5rbRk', '5t4rndvpGfblr16xsSZCZ2', '05HbL11gtNMrCPFadpwoRg', '4de8RbgaXo4j0skA36mPU2', '1tPOHHQY0x7uUNAHnwTjlx', '3mUM0957VDc8b5ynZr1Gpn', '5Q2yV1wgKENv1XsCTB7nma', '6NQXCldnycOtVhlh33uHRr', '35o92ikbXQApq9xBjLgbnV', '596eSWS6SoVNLoe9h1CMXn', '0ClP3NmCW8WBitkdFqKJ0x', '0RtxITcYXbdcoDXhwAgNWd', '6F7nb8f7ty7P4vLQeFL1lq', '6Mc4X5zjF3GtHPvIrs14fe', '1dLUGmTp8Fy9Sl0p2Tn7T1', '5is0Z9t8qzQt3bWFgbhYTw', '4m1h2nXCfcDfRY3WPY21ko', '66IrwRUKnmMPiYpBfEGYCm', '2GXA1T0GBh3pXZMTA50OhL', '0pwObEOHolQZSldJ2q1wpy']
Track IDs: ['6AioOohg4bQZFA4jIYQQ2r', '1lATXTBJDHwawvT1UfxWu3', '6DyywdbmTzlmXBzG9ym7Rt', '55mJleti2WfWEFNFcBduhc', '2hnrioxoZCljW6VbbO0Bvv', '54zCGbgOeaOXsEFLgLuwzB', '7upG8kgnfF4yfaOGNKHXJg', '7anlKmDzahsUb5dlDhrYYr', '53Gs2a7kV4h6tBN0fCTtUY', '3ZWPE72rn7gB2uHFe5s0HJ', '4nrdKOChOn750sd3tI40OE', '0UqwIL59AsdSTdmewiWXvj', '09fNyeCFUbrTor4DY7kX2b', '7GbNB9IGxXN9QiX8x83TXI', '07aCEOE0LXRa0cxdrrpWlH', '3MZgRR5kAyMcgnn3LuhgQK', '6xnl08BI56Ej3dnGRqc6r

In [4]:
print(track_data['genres'])
print(len(track_data['id']))

[['album rock', 'classic rock', 'mellow gold', 'rock', 'singer-songwriter', 'soft rock', 'yacht rock'], ['puerto rican indie', 'tropical alternativo'], ['hypnagogic pop'], ['slacker rock'], [], [], [], ['indie punk', 'indie rock', 'philly indie'], [], [], [], [], ['latin afrobeat', 'latintronica'], ['tropical alternativo'], ['tropical alternativo'], [], ['indie folk argentino'], ['instrumental funk'], [], ['alternative metal', 'art rock', 'nu metal', 'post-grunge', 'progressive metal', 'progressive rock', 'rock'], ['alternative metal', 'art rock', 'nu metal', 'post-grunge', 'progressive metal', 'progressive rock', 'rock'], ['alternative metal', 'art rock', 'nu metal', 'post-grunge', 'progressive metal', 'progressive rock', 'rock'], ['alternative metal', 'art rock', 'nu metal', 'post-grunge', 'progressive metal', 'progressive rock', 'rock'], ['alternative metal', 'art rock', 'nu metal', 'post-grunge', 'progressive metal', 'progressive rock', 'rock'], ['j-rock', 'japanese alternative roc

In [5]:
genre_dict= dict(zip(track_data['id'],track_data['genres']))
print(genre_dict)


{'4o6BgsqLIBViaGVbx5rbRk': ['album rock', 'classic rock', 'mellow gold', 'rock', 'singer-songwriter', 'soft rock', 'yacht rock'], '5t4rndvpGfblr16xsSZCZ2': ['puerto rican indie', 'tropical alternativo'], '05HbL11gtNMrCPFadpwoRg': ['hypnagogic pop'], '4de8RbgaXo4j0skA36mPU2': ['slacker rock'], '1tPOHHQY0x7uUNAHnwTjlx': [], '3mUM0957VDc8b5ynZr1Gpn': [], '5Q2yV1wgKENv1XsCTB7nma': [], '6NQXCldnycOtVhlh33uHRr': ['indie punk', 'indie rock', 'philly indie'], '35o92ikbXQApq9xBjLgbnV': [], '596eSWS6SoVNLoe9h1CMXn': [], '0ClP3NmCW8WBitkdFqKJ0x': [], '0RtxITcYXbdcoDXhwAgNWd': [], '6F7nb8f7ty7P4vLQeFL1lq': ['latin afrobeat', 'latintronica'], '6Mc4X5zjF3GtHPvIrs14fe': ['tropical alternativo'], '1dLUGmTp8Fy9Sl0p2Tn7T1': ['tropical alternativo'], '5is0Z9t8qzQt3bWFgbhYTw': [], '4m1h2nXCfcDfRY3WPY21ko': ['indie folk argentino'], '66IrwRUKnmMPiYpBfEGYCm': ['instrumental funk'], '2GXA1T0GBh3pXZMTA50OhL': [], '0pwObEOHolQZSldJ2q1wpy': ['alternative metal', 'art rock', 'nu metal', 'post-grunge', 'progressi

In [6]:
genre_list=[]
for song in genre_dict:
    for genre in genre_dict[song]:
        if genre not in genre_list:
            genre_list.append(genre)

genre_list.append('no genre')
print(genre_list)

['album rock', 'classic rock', 'mellow gold', 'rock', 'singer-songwriter', 'soft rock', 'yacht rock', 'puerto rican indie', 'tropical alternativo', 'hypnagogic pop', 'slacker rock', 'indie punk', 'indie rock', 'philly indie', 'latin afrobeat', 'latintronica', 'indie folk argentino', 'instrumental funk', 'alternative metal', 'art rock', 'nu metal', 'post-grunge', 'progressive metal', 'progressive rock', 'j-rock', 'japanese alternative rock', 'j-indie', 'japanese garage rock', 'japanese emo', 'japanese post-rock', 'japanese shoegaze', 'japanese indie pop', 'japanese dream pop', 'japanese indie rock', 'kansai indie', 'canadian psychedelic rock', 'neo-psychedelic', 'psych gaze', 'german stoner rock', 'space rock', 'stoner metal', 'stoner rock', 'scottish shoegaze', 'uk stoner rock', 'british folk', 'british invasion', 'folk', 'folk rock', 'glam rock', 'psychedelic folk', 'psychedelic rock', 'scottish singer-songwriter', 'no genre']


In [7]:
playlist_dict={}
no_genre_songs=[]
for genre in genre_list:
    song_list=[]
    for song in genre_dict:
        if genre in genre_dict[song] and song not in song_list:
            song_list.append(song)
        if not genre_dict[song] and song not in no_genre_songs:
            no_genre_songs.append(song)

    playlist_dict[genre]=song_list

playlist_dict['no genre']=no_genre_songs


print(len(list(playlist_dict.keys())))
print(playlist_dict)
print(len(playlist_dict['no genre']))


53
{'album rock': ['4o6BgsqLIBViaGVbx5rbRk'], 'classic rock': ['4o6BgsqLIBViaGVbx5rbRk', '4vYvv6ODTNaEn3Do2CFpCc'], 'mellow gold': ['4o6BgsqLIBViaGVbx5rbRk'], 'rock': ['4o6BgsqLIBViaGVbx5rbRk', '0pwObEOHolQZSldJ2q1wpy', '6AioOohg4bQZFA4jIYQQ2r', '1lATXTBJDHwawvT1UfxWu3', '6DyywdbmTzlmXBzG9ym7Rt', '55mJleti2WfWEFNFcBduhc'], 'singer-songwriter': ['4o6BgsqLIBViaGVbx5rbRk', '4vYvv6ODTNaEn3Do2CFpCc'], 'soft rock': ['4o6BgsqLIBViaGVbx5rbRk'], 'yacht rock': ['4o6BgsqLIBViaGVbx5rbRk'], 'puerto rican indie': ['5t4rndvpGfblr16xsSZCZ2'], 'tropical alternativo': ['5t4rndvpGfblr16xsSZCZ2', '6Mc4X5zjF3GtHPvIrs14fe', '1dLUGmTp8Fy9Sl0p2Tn7T1'], 'hypnagogic pop': ['05HbL11gtNMrCPFadpwoRg'], 'slacker rock': ['4de8RbgaXo4j0skA36mPU2'], 'indie punk': ['6NQXCldnycOtVhlh33uHRr'], 'indie rock': ['6NQXCldnycOtVhlh33uHRr'], 'philly indie': ['6NQXCldnycOtVhlh33uHRr'], 'latin afrobeat': ['6F7nb8f7ty7P4vLQeFL1lq'], 'latintronica': ['6F7nb8f7ty7P4vLQeFL1lq'], 'indie folk argentino': ['4m1h2nXCfcDfRY3WPY21ko'], 'in

In [8]:
batch_size=50
user_playlists=[]

for offset in range(0, 9000, batch_size):
    user_playlists_batch=sp.user_playlists(user=target_user_id,limit=batch_size,offset=offset)
    user_playlists += user_playlists_batch["items"]
    print(len(user_playlists_batch["items"]))
    if len(user_playlists_batch["items"]) < batch_size:
        break

50
50
50
11


In [9]:
uplaylist_test=sp.user_playlists(user=target_user_id)
print(type(user_playlists_batch['items']))
print(len(uplaylist_test))
print(type(uplaylist_test))
print(len(uplaylist_test["items"]))
#print(json.dumps(uplaylist_test,indent=2))
print(len(user_playlists))
print(json.dumps(user_playlists[0],indent=2))
#ignore this cell

<class 'list'>
7
<class 'dict'>
50
161
{
  "collaborative": false,
  "description": "",
  "external_urls": {
    "spotify": "https://open.spotify.com/playlist/6ELXc3NZxHrXBKZhD2vBb5"
  },
  "href": "https://api.spotify.com/v1/playlists/6ELXc3NZxHrXBKZhD2vBb5",
  "id": "6ELXc3NZxHrXBKZhD2vBb5",
  "images": [
    {
      "height": 640,
      "url": "https://mosaic.scdn.co/640/ab67616d00001e020e03e06e3dab5d79175206dbab67616d00001e02618e438409038bcd054d1ca7ab67616d00001e029dbaf7f48c95b0d32194b56fab67616d00001e02af9652377659e3edbbcd5cdc",
      "width": 640
    },
    {
      "height": 300,
      "url": "https://mosaic.scdn.co/300/ab67616d00001e020e03e06e3dab5d79175206dbab67616d00001e02618e438409038bcd054d1ca7ab67616d00001e029dbaf7f48c95b0d32194b56fab67616d00001e02af9652377659e3edbbcd5cdc",
      "width": 300
    },
    {
      "height": 60,
      "url": "https://mosaic.scdn.co/60/ab67616d00001e020e03e06e3dab5d79175206dbab67616d00001e02618e438409038bcd054d1ca7ab67616d00001e029dbaf7f48c95b0d

In [10]:
for key in list(playlist_dict.keys()):
    for playlist in user_playlists:
        if playlist['name'] == key:
            x=sp.user_playlist_unfollow(user=target_user_id,playlist_id=playlist['id']) 
            #unfollow cleans the account from any playlist with the same name

    playlist_created=sp.user_playlist_create(user=target_user_id,name=key,public=True,collaborative=False,description='')
    #creates the playlist for the name 'key'

    for songs in playlist_dict[key]:
        playlist_songlist=sp.user_playlist_add_tracks(user=target_user_id, playlist_id=playlist_created['id'], tracks=playlist_dict[key], position=None)
    #adds songs saved in the genre-songs dictionary in the playlist created before